In [205]:
import copy

In [206]:
def Construct_Left(shape):
    left_array = []
    for i in range(len(shape)):
        left_array.append([])
        for j in range(shape[i]):
            left_array[i].append(0)
    return left_array

def Construct_Right(shape):
    left_array = []
    for i in range(len(shape)):
        left_array.append([])
        for j in range(shape[i]):
            left_array[i].append(i+1)
    return left_array

def Count_Init(shape):
    left_array = []
    for i in range(len(shape)):
        left_array.append([])
        for j in range(shape[i]):
            left_array[i].append(0)
    return left_array

def print_array(array, s):
    print("Array " + s + ":")
    for i in range(len(array)):
        for j in range(len(array[i])):
            print(array[i][j], end='\t')
        print()
    print()

In [275]:
def solve(shape_left, shape_right, max_line):    
    # Building States
    current_array = Construct_Left(shape_left)
    current_shape = shape_left
    current_number = 1
    remaining_current_number = shape_right[0]
    current_putting_line = 0
    forbidden_columns_set = set()
    previous_number_count = []
    current_number_count = Count_Init(current_shape)
    
    # Counter of the number of the answers
    count = 0
    
    # A dictionary whose key is the shape of the answer and whose value is the number of results corresponding to its shape
    answer_dict = {}
    
    # Biggest number of the right array
    last_number = len(shape_right)
    
    # Initialize the stack
    stack = []
    init_dict = {"current_array": current_array, "current_shape": current_shape, "current_number": current_number, 
                 "remaining_current_number":remaining_current_number, "current_putting_line": current_putting_line, 
                 "forbidden_columns_set":forbidden_columns_set, "previous_number_count": previous_number_count,
                "current_number_count": current_number_count}
    
    stack.append(init_dict)
    
    # Repeatly pop the stack and process the state poped until it is empty
    while (len(stack) != 0):
        # Pop the stack and update the state-related variables
        current_state = stack.pop()
        current_array = copy.deepcopy(current_state["current_array"])
        current_shape = copy.deepcopy(current_state["current_shape"])
        current_number = current_state["current_number"]
        remaining_current_number = current_state["remaining_current_number"]
        current_putting_line = current_state["current_putting_line"]
        forbidden_columns_set = copy.deepcopy(current_state["forbidden_columns_set"])
        previous_number_count = copy.deepcopy(current_state["previous_number_count"])
        current_number_count = copy.deepcopy(current_state["current_number_count"])

        # If we have found a solution, that is to say we have put all blocks to the left array, we count it and output the result
        if ((current_number == last_number) and (remaining_current_number == 0)):
            count += 1
            current_shape_tuple = tuple(current_shape)
            if (current_shape_tuple in answer_dict):
                answer_dict[current_shape_tuple] += 1
            else:
                answer_dict[current_shape_tuple] = 1
            print_array(current_array, str(count))
            continue
        
        # If we have finished one type of the boxes, we continue with the next type (states updated)
        if ((current_number < last_number) and (remaining_current_number == 0)):
            remaining_current_number = shape_right[current_number]
            current_number += 1
            current_putting_line = 0
            forbidden_columns_set = set()
            previous_number_count = current_number_count
            current_number_count = Count_Init(current_shape)
            stack.append({"current_array": current_array, "current_shape": current_shape, "current_number": current_number, 
                 "remaining_current_number":remaining_current_number, "current_putting_line": current_putting_line, 
                 "forbidden_columns_set":forbidden_columns_set, "previous_number_count": previous_number_count,
                "current_number_count": current_number_count})
            continue
            
        # We try to put the new box of current_number into different lines
        for new_box_line in range(current_putting_line, max_line):
            current_array = copy.deepcopy(current_state["current_array"])
            current_shape = copy.deepcopy(current_state["current_shape"])
            current_number = current_state["current_number"]
            remaining_current_number = current_state["remaining_current_number"]
            current_putting_line = current_state["current_putting_line"]
            forbidden_columns_set = copy.deepcopy(current_state["forbidden_columns_set"])
            previous_number_count = copy.deepcopy(current_state["previous_number_count"])
            current_number_count = copy.deepcopy(current_state["current_number_count"])

            # If we have reached two lines after the last line, we break due to Rule 2
            if (new_box_line > len(current_array)):
                break
            
            # To check whether it is necessary to add a new line
            if (new_box_line == len(current_array)):
                current_array.append([])
                current_shape.append(0)
                if (current_number != 1):
                    previous_number_count.append([])
                current_number_count.append([])
            
            # Put in the new box and update all states
            current_array[new_box_line].append(current_number)
            current_shape[new_box_line] += 1
            remaining_current_number -= 1
            current_putting_line = new_box_line
            if (current_number != 1):
                if (new_box_line == 0):
                    previous_number_count[new_box_line].append(0)
                elif (len(previous_number_count[new_box_line]) != 0):
                    previous_number_count[new_box_line].append(previous_number_count[new_box_line][-1])
                else:
                    previous_number_count[new_box_line].append(previous_number_count[new_box_line-1][0])
            if (new_box_line == 0):
                current_number_count[new_box_line].append(0)
            elif (len(current_number_count[new_box_line]) != 0):
                current_number_count[new_box_line].append(current_number_count[new_box_line][-1])
            else:
                current_number_count[new_box_line].append(current_number_count[new_box_line-1][0])
            for i in range(new_box_line, len(current_shape)):
                current_number_count[i] = [t+1 for t in current_number_count[i]]
            
            # We check four rules sequentially to check whether the current state is valid or not.
            # If the current state is invalid, then there is no need to continue, since according to the process, an invalid state will not become valid.
            # Rule 1: Number of boxes in column n+1 is less than that in column n
            # This rule can not be broken if each new box is built following the last box in the line
            
            # Rule 2: Number of boxes in line n+1 is less than that in line n
            if (new_box_line != 0):
                new_box_line_num = len(current_array[new_box_line])
                previous_line_num = len(current_array[new_box_line-1])
                if (new_box_line_num > previous_line_num):
                    continue
            
            # Rule 3: The boxes of the same index cannot appear twice or more in one colume
            new_box_column = len(current_array[new_box_line]) - 1
            if (new_box_column in forbidden_columns_set):
                continue
            else:
                forbidden_columns_set.add(new_box_column)
                
            # Rule 4: Count from right to left, from up to down
            # The count process has been executed each time when we add a new box, so we only need to compare the number.
            flag = True
            if (current_number != 1):
                for i in range(len(current_shape)):
                    if (not flag):
                        break
                    for j in range(current_shape[i]):
                        if (previous_number_count[i][j] < current_number_count[i][j]):
                            flag = False
                            break
            if (not flag):
                continue
                        
            stack.append({"current_array": current_array, "current_shape": current_shape, "current_number": current_number, 
                 "remaining_current_number":remaining_current_number, "current_putting_line": current_putting_line, 
                 "forbidden_columns_set":forbidden_columns_set, "previous_number_count": previous_number_count,
                "current_number_count": current_number_count})
    
    return count, answer_dict

In [277]:
shape_left = [5,5,5,5,5]
shape_right = [4,3,2,1]

left_array = Construct_Left(shape_left)
print_array(left_array, "left")

right_array = Construct_Right(shape_right)
print_array(right_array, "right")

solve(shape_left=shape_left, shape_right=shape_right, max_line=3)

Array left:
0	0	0	0	0	
0	0	0	0	0	
0	0	0	0	0	
0	0	0	0	0	
0	0	0	0	0	

Array right:
1	1	1	1	
2	2	2	
3	3	
4	

Array 1:
0	0	0	0	0	1	1	1	1	
0	0	0	0	0	2	2	2	
0	0	0	0	0	3	3	4	
0	0	0	0	0	
0	0	0	0	0	

Array 2:
0	0	0	0	0	1	1	1	1	
0	0	0	0	0	2	2	2	3	
0	0	0	0	0	3	4	
0	0	0	0	0	
0	0	0	0	0	



(2, {(9, 8, 8, 5, 5): 1, (9, 9, 7, 5, 5): 1})